In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/house-price-prediction-challenge/train.csv")
df_test = pd.read_csv("/kaggle/input/house-price-prediction-challenge/test.csv")
print(df_train.shape)
print(df_test.shape)

In [ ]:
df_train.info()

In [ ]:
df_train.head(10)

In [ ]:
print(df_train['POSTED_BY'].unique())
print(df_train['BHK_OR_RK'].unique())

In [ ]:
print("\n\nBHK_NO.: ")
print(df_train['BHK_NO.'].value_counts())

print("\n\nBHK_NO.: ")
print(df_train['POSTED_BY'].value_counts())

print("\n\nBHK_NO.: ")
print(df_train['READY_TO_MOVE'].value_counts())

print("\n\nBHK_NO.: ")
print(df_train['UNDER_CONSTRUCTION'].value_counts())

In [ ]:
metro_cities = ['Ahmedabad', 'Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Kolkata', 'Mumbai', 'Pune', 'Gurgaon', 'Chandigarh']

In [ ]:
df_train['city'] = df_train['ADDRESS'].str.split(",").str[-1]
df_test['city'] = df_test['ADDRESS'].str.split(",").str[-1]

In [ ]:
df_train.head(5)

In [ ]:
df_test.head(5)

In [ ]:
def set_metro_city(row, metro_city):
    if metro_city == 1:
        if row['city'] in metro_cities:
            return 1
        else:
            return 0

df_train['metro_city'] = df_train.apply(set_metro_city, args=([1]), axis=1)
df_test['metro_city'] = df_test.apply(set_metro_city, args=([1]), axis=1)

In [ ]:
df_train.head(10)

In [ ]:
df_test.head(10)

In [ ]:
df_train.drop(['ADDRESS','city'], axis = 1, inplace = True)
print(df_train.shape)
df_test.drop(['ADDRESS', 'city'], axis = 1, inplace = True)
print(df_test.shape)

In [ ]:

df_train = pd.get_dummies(df_train, columns=["POSTED_BY", 'BHK_OR_RK'], prefix=["POSTED_BY", 'IS_'] )
df_test = pd.get_dummies(df_test, columns=["POSTED_BY", 'BHK_OR_RK'], prefix=["POSTED_BY", 'IS_'] )


In [ ]:
df_train.head(10)

In [ ]:
df_test.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

#train, test = train_test_split(df_features_selected,test_size=0.20, random_state=0)
train, test = train_test_split(df_train,test_size=0.20, random_state=0, stratify=df_train[['READY_TO_MOVE', 'RESALE', 'UNDER_CONSTRUCTION', 'RERA']])

X_train = train[train.columns.difference(['TARGET(PRICE_IN_LACS)'])]
y_train = train['TARGET(PRICE_IN_LACS)']

X_test = test[test.columns.difference(['TARGET(PRICE_IN_LACS)'])]
y_test = test['TARGET(PRICE_IN_LACS)']

print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rf_model = RandomForestRegressor(n_estimators=200, random_state=42)

rf_model.fit(X_train, y_train)

preds = rf_model.predict(X_test)

print('Random Forest: ', r2_score(y_test, preds))

In [ ]:
from xgboost import XGBRegressor

xgboost_model = XGBRegressor(n_estimators=100, learning_rate=0.05, random_state=42)

xgboost_model.fit(X_train, y_train)

preds = xgboost_model.predict(X_test)

print('XG Boost: ', r2_score(y_test, preds))

In [ ]:
df_test_output = xgboost_model.predict(df_test)

# Save to submission.csv file
df_output = pd.DataFrame({'Id': df_test.index, 'PredictedSalePrice': df_test_output})
df_output.to_csv('submission_xgb.csv', index=False)
df_output.head()